In [1]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import dotenv
import os 
import accelerate

/opt/homebrew/Caskroom/mambaforge/base/envs/hertel_pc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


import pandas as pd
import json
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset


model_name = "serlikopar/Enlighten_Instruct"

In [2]:
dotenv.load_dotenv("hugging_face.env")

True

In [3]:
secret_hf = os.getenv('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/serlink/.cache/huggingface/token
Login successful


In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "serlikopar/Enlighten_Instruct"
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)
model = PeftModel.from_pretrained(base_model_reload, new_model)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.75s/it]
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'Mistral

In [7]:
import pandas as pd
import json

def make_df_from_jsonl(file_path):
    questions = []
    answers = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                json_data = json.loads(line)
                # Extract messages
                messages = json_data['messages']
                # Initialize placeholders for user and assistant messages
                question = next((msg['content'] for msg in messages if msg['role'] == 'user'), None)
                answer = next((msg['content'] for msg in messages if msg['role'] == 'assistant'), None)
                
                # Only append if both question and answer are found
                if question and answer:
                    questions.append(question.replace('\\n', '\n'))
                    answers.append(answer.replace('\\n', '\n'))
                
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON on line {line_number}: {e}")
                continue

    # Create DataFrame with specific columns
    df = pd.DataFrame({'Question': questions, 'Answer': answers})
    
    return df




In [29]:
file_path = 'test_samples.jsonl'
df = make_df_from_jsonl(file_path)

In [30]:
df

,Question,Answer
0,Was besagt die Zustandsgleichung des idealen G...,Die Zustandsgleichung des idealen Gases lautet...
1,Was ist ein ideales Gas?,Ein ideales Gas entspricht einer Modellvorstel...
2,Was besagt der Nullte Hauptsatz der Thermodyna...,"Der Nullte Hauptsatz der Thermodynamik besagt,..."
3,Was ist der Partialdruck?,Der Partialdruck einer Komponente in einem Gas...
4,Wie lautet das Amontonssche Gesetz?,"Das Amontonssche Gesetz besagt, dass der Druck..."
...,...,...
57,Warum wird die molare Wärmekapazität in J/(mol...,Die molare Wärmekapazität wird in J/(mol·K) an...
58,Warum ist die kinetische Energie der Gasteilch...,Die kinetische Energie der Gasteilchen ist tem...
59,Warum beschreibt die van-der-Waals-Gleichung r...,Die van-der-Waals-Gleichung beschreibt reale G...
60,"Warum erhöht sich der Druck eines Gases, wenn ...","Der Druck eines Gases erhöht sich, wenn mehr M..."


In [31]:
df.head()

,Question,Answer
0,Was besagt die Zustandsgleichung des idealen G...,Die Zustandsgleichung des idealen Gases lautet...
1,Was ist ein ideales Gas?,Ein ideales Gas entspricht einer Modellvorstel...
2,Was besagt der Nullte Hauptsatz der Thermodyna...,"Der Nullte Hauptsatz der Thermodynamik besagt,..."
3,Was ist der Partialdruck?,Der Partialdruck einer Komponente in einem Gas...
4,Wie lautet das Amontonssche Gesetz?,"Das Amontonssche Gesetz besagt, dass der Druck..."


In [32]:
df.columns

Index(['Question', 'Answer'], dtype='object')

In [33]:
df.to_csv("test_data_62_questions_with_formatting.csv", index=False)

In [34]:
df['text'] = '<s>[INST]' + df['Question'] +'[/INST]'+ df['Answer'] + '</s>'



In [35]:
df

,Question,Answer,text
0,Was besagt die Zustandsgleichung des idealen G...,Die Zustandsgleichung des idealen Gases lautet...,<s>[INST]Was besagt die Zustandsgleichung des ...
1,Was ist ein ideales Gas?,Ein ideales Gas entspricht einer Modellvorstel...,<s>[INST]Was ist ein ideales Gas?[/INST]Ein id...
2,Was besagt der Nullte Hauptsatz der Thermodyna...,"Der Nullte Hauptsatz der Thermodynamik besagt,...",<s>[INST]Was besagt der Nullte Hauptsatz der T...
3,Was ist der Partialdruck?,Der Partialdruck einer Komponente in einem Gas...,<s>[INST]Was ist der Partialdruck?[/INST]Der P...
4,Wie lautet das Amontonssche Gesetz?,"Das Amontonssche Gesetz besagt, dass der Druck...",<s>[INST]Wie lautet das Amontonssche Gesetz?[/...
...,...,...,...
57,Warum wird die molare Wärmekapazität in J/(mol...,Die molare Wärmekapazität wird in J/(mol·K) an...,<s>[INST]Warum wird die molare Wärmekapazität ...
58,Warum ist die kinetische Energie der Gasteilch...,Die kinetische Energie der Gasteilchen ist tem...,<s>[INST]Warum ist die kinetische Energie der ...
59,Warum beschreibt die van-der-Waals-Gleichung r...,Die van-der-Waals-Gleichung beschreibt reale G...,<s>[INST]Warum beschreibt die van-der-Waals-Gl...
60,"Warum erhöht sich der Druck eines Gases, wenn ...","Der Druck eines Gases erhöht sich, wenn mehr M...",<s>[INST]Warum erhöht sich der Druck eines Gas...


In [36]:
df2 = df["text"]

In [37]:
df2 = pd.DataFrame(df2)

In [38]:
type(df2)

pandas.core.frame.DataFrame

In [39]:
df2

,text
0,<s>[INST]Was besagt die Zustandsgleichung des ...
1,<s>[INST]Was ist ein ideales Gas?[/INST]Ein id...
2,<s>[INST]Was besagt der Nullte Hauptsatz der T...
3,<s>[INST]Was ist der Partialdruck?[/INST]Der P...
4,<s>[INST]Wie lautet das Amontonssche Gesetz?[/...
...,...
57,<s>[INST]Warum wird die molare Wärmekapazität ...
58,<s>[INST]Warum ist die kinetische Energie der ...
59,<s>[INST]Warum beschreibt die van-der-Waals-Gl...
60,<s>[INST]Warum erhöht sich der Druck eines Gas...


In [40]:
df2.to_csv("test_data_62_questions_one_col.csv", index=False)